In [17]:
import os
import pandas as pd
import pmdarima as pm
from pmdarima import model_selection
from sklearn.metrics import mean_squared_error
import numpy as np
import warnings
import time

# Hyndman-Khandakar algorithm
1. Determine d using repeated stationarity tests.
2. choose p and q, by minimizing the AICcs.

## no differentiation, d = 0

In [18]:
warnings.filterwarnings("ignore")

def hyndman_khandakar_non_seasonal_forecast(time_series: pd.Series, dif : int, p: int, q: int):

    print(f"Arima (Hyndman-Khandakar logic)")
    
    model = pm.auto_arima(
        y = time_series,
        start_p = 1, start_q = 1,
        max_p = p, max_q = q,
        seasonal = False,
        maxiter = 15,
        d = dif,
        test = "kpss",
        stepwise = True,
        trace = False,
        error_action = 'trace',
        suppress_warnings = True,
        information_criterion = 'aicc'
    )

    print(f"\nBest Model")
    print(model.summary())
    print("-" * 40)
    print(f"Selected ARIMA Order: {model.order}")

    return model


In [19]:
def forecasting_HK(model, h):
    forecasts = model.predict(h)
    
    if isinstance(time_series, pd.Series):
        try:
            last_date = time_series.index[-1]
            if isinstance(last_date, pd.Timestamp):
                 # For datetime index
                freq = time_series.index.freq if time_series.index.freq is not None else pd.infer_freq(time_series.index)
                forecast_index = pd.date_range(start=last_date, periods=h + 1, freq=freq)[1:]
            else:
                 # For a simple integer index
                forecast_index = pd.RangeIndex(start=len(time_series), stop=len(time_series) + h)
        except Exception:
            forecast_index = pd.RangeIndex(start=len(time_series), stop=len(time_series) + h)


    forecast_series = pd.Series(forecasts, index=forecast_index, name='Forecast')
    print(f"forecast series : {forecast_series}")
    return forecast_series


In [20]:
def cast_to_time(df):
    df['time'] = pd.to_datetime(df['time'])
    return df

In [21]:
data_dir = '../Data/Preprocessed'
base_paths = os.listdir(data_dir)
full_paths = [os.path.join(data_dir, p) for p in base_paths]
patients = {base_paths[i].split('.')[0] : cast_to_time(pd.read_csv(fp, sep=';')) for i, fp in enumerate(full_paths)}
ids = list(patients.keys())
s = 1

In [22]:
too_long = ["HUPA0027P", "HUPA0028P", "HUPA0026P"]
for i in too_long:
    ids.remove(i)

In [23]:
results_d0 = []

for i in ids:
    d = 0
    ts = patients[i].loc[:, "glucose"]
    print(f"longitud : {len(ts)}")
    print(f"paciente : {i}")
    # Run the non-seasonal HK algorithm
    result = hyndman_khandakar_non_seasonal_forecast(time_series = ts, dif = d, p = 6, q = 6)
    results_d0.append(result)
    time.sleep(s)


longitud : 3895
paciente : HUPA0018P
Arima (Hyndman-Khandakar logic)

Best Model
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                 3895
Model:               SARIMAX(5, 0, 4)   Log Likelihood               -8444.041
Date:                Mon, 06 Oct 2025   AIC                          16910.082
Time:                        14:48:19   BIC                          16979.024
Sample:                             0   HQIC                         16934.553
                               - 3895                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.5428      1.758      0.309      0.757      -2.903       3.988
ar.L1          1.7587      0.987      1.782      0

In [24]:
results_d1 = []

for i in ids:
    d = 1
    ts = patients[i].loc[:, "glucose"]
    print(f"longitud : {len(ts)}")
    print(f"paciente : {i}")
    # Run the non-seasonal HK algorithm
    result = hyndman_khandakar_non_seasonal_forecast(time_series = ts, dif = d, p = 5, q = 5)
    results_d1.append(result)
    time.sleep(s)


longitud : 3895
paciente : HUPA0018P
Arima (Hyndman-Khandakar logic)

Best Model
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                 3895
Model:               SARIMAX(3, 1, 3)   Log Likelihood               -8490.792
Date:                Mon, 06 Oct 2025   AIC                          16995.584
Time:                        14:54:04   BIC                          17039.454
Sample:                             0   HQIC                         17011.156
                               - 3895                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.0097      0.172      0.057      0.955      -0.327       0.346
ar.L2          0.0525      0.090      0.581      0